# Visualizations Notebook

The following notebook has as a purpose to show visualizations regarding the Education status at South America.

### Set up of libraries

In [1]:
import pandas as pd
import altair as alt

In [2]:
from utils import filters_countries_cols, translate_english_degrees
from utils import translate_area_del_conocimiento, translates_degree

In [3]:
# Loading the data
raw = pd.read_csv("../data/raw/matricula.csv", encoding="latin1", sep=None, engine="python")


In [4]:
# year column to number
raw["AÑO"] = raw["AÑO"].astype(str).str[-4:]
raw["AÑO"] = raw["AÑO"].astype(int)

# columns in lowercase
cols = [col.lower() for col in raw.columns]
raw.columns = cols

#

In [5]:
raw["carrera clasificación nivel 2"].unique()

array(['Carreras Técnicas', 'Doctorado', 'Carreras Profesionales',
       'Postítulo', 'Magister'], dtype=object)

# **First Chart: Evolution total students of undergrads, and grad students 2007-2025**

This line chart shows the total number of enrolled students for undergrads, grads and diploma students
in Chile for the period 2007-2025

In [6]:
# Getting the data
cols_1 = ["año", "nivel global"]
df_1 = raw.groupby(cols_1)["total matrícula"].sum().reset_index() 
df_1["nivel global"] = df_1.apply(translate_english_degrees, axis=1)
df_1.rename(columns = {"nivel global" : "Degree Type"}, inplace=True) 

# Chart number of enrolled during type
chart_1 = alt.Chart(df_1).mark_area().encode(
    x=alt.Y("año:O", axis= alt.Axis(title="Year"), scale=alt.Scale(2007,2025)),
    y=alt.Y("total matrícula:Q", axis= alt.Axis(title="Total enrolled")),
    color = "Degree Type:N").properties(
    title = "Total enrolled students by Degree Type for 2007-2025"
)

chart_1

alt.Chart(...)

# **Second Chart: Out of undergrads, relation of professional and technical undergrads**

The following chart shows the relation out of undergrads of students studying
technical careers and professional careers


In [7]:
# Degrees to show


degrees = ["Carreras Profesionales", "Carreras Técnicas"]

# Data
cols_2 = ["año", "carrera clasificación nivel 2"]
df_2 = raw.groupby(cols_2)["total matrícula"].sum().reset_index()

# Cleaning the data
#df_2.rename(columns={"carrera clasificación nivel 2": "Degree type of undergrads"}, inplace=True)
df_2 = df_2[df_2["carrera clasificación nivel 2"].isin(degrees)]
df_2["carrera clasificación nivel 2"] = df_2.apply(translates_degree, axis=1)

chart_2 = alt.Chart(df_2).mark_bar().encode(
    x = alt.X("año:O", axis=alt.Axis(title="Year")),
    y = alt.Y("sum(total matrícula)", axis=alt.Axis(title="Total enrolled")).stack("normalize"),
    color = alt.Color("carrera clasificación nivel 2",
                      legend = alt.Legend(title="Degree type of undergrads"))).properties(
        title="Share of Technical and Professioanl enrolled undergrads"
    )

chart_2


alt.Chart(...)

# **Third Chart: Evolution Total male and female students of overall enrolled**

The following chart shows the growth of male and females students during 2007-2025


In [8]:
cols_3 = ["año"]
df_3 = raw.groupby(cols_3)[["total matrícula hombres", "total matrícula mujeres"]].sum().reset_index()



df_3 = df_3.melt(id_vars="año",
                 value_vars=["total matrícula hombres", "total matrícula mujeres"],
                 var_name="enrolled type",
                 value_name="total matrícula")

men = "total matrícula hombres"
women = "total matrícula mujeres"

df_3["enrolled type"] = df_3["enrolled type"].apply(lambda x: "Men" if x == men else "Women")

chart_3 = alt.Chart(df_3).mark_line().encode(
    x = alt.X("año:O", axis=alt.Axis(title="Year")),
    y = alt.Y("sum(total matrícula)", axis=alt.Axis(title="Total enrolled")),
    color = alt.Color("enrolled type",
                      legend=alt.Legend(title="Gender"))).properties(
        title = "Evolution of enrolled Men and Women in universities' Degrees 2007-2025"
    )

chart_3


alt.Chart(...)

# **Fourth Chart: Differences of study areas between men and women**

The following chart shows the ranking of top enrolled number of students of
knowkledge areas, comparing men and women for the 2007-2025 period

In [9]:
# For Women

cols_4 = ["año", "área del conocimiento"]
col_add = "total matrícula mujeres"
df_4 = raw.groupby(cols_4)[col_add].sum().reset_index()

df_4["área del conocimiento"] = df_4.apply(translate_area_del_conocimiento, axis=1)

df_4_final = pd.DataFrame({"año": [], "área del conocimiento": [],
                           "total matrícula mujeres": [], "rank": []})

for year in df_4["año"].unique():
    yr_df = df_4[df_4["año"] == year].copy()
    yr_df["rank"] = yr_df["total matrícula mujeres"].rank(ascending=False)
    df_4_final = pd.concat([df_4_final, yr_df])

chart_4 = alt.Chart(df_4_final).mark_line().encode(
    x = alt.X("año:O").title("Year"),
    y = alt.Y("rank").title("Rank"),
    color = alt.Color("área del conocimiento",
                      legend = alt.Legend(title="Study Field"))).properties(
                          title = "Study field preference by WOMEN"
                      )

In [10]:
# For Men

cols_44 = ["año", "área del conocimiento"]
col_add_44 = "total matrícula hombres"
df_44 = raw.groupby(cols_44)[col_add_44].sum().reset_index()

df_44["área del conocimiento"] = df_44.apply(translate_area_del_conocimiento, axis=1)

df_44_final = pd.DataFrame({"año": [], "área del conocimiento": [],
                           "total matrícula hombres": [], "rank": []})

for year in df_44["año"].unique():
    yr_df = df_44[df_44["año"] == year].copy()
    yr_df["rank"] = yr_df["total matrícula hombres"].rank(ascending=False)
    df_44_final = pd.concat([df_44_final, yr_df])


chart_44 = alt.Chart(df_44_final).mark_line().encode(
    x = alt.X("año:O"),
    y = "rank",
    color = alt.Color("área del conocimiento",
                      legend = alt.Legend(title="Study Field"))).properties(
                          title = "Study field preference by MEN")
chart_44

final = chart_4 | chart_44
final

alt.HConcatChart(...)

# **Fifth Chart: Where in Chile are they studying at 2025**

The following Chart shows the number of univerisity student in each region of
Chile ordered from north to South at 2025

In [11]:
cols_5 = ["año", "región"]
value_5 = "total matrícula"
df_5 = raw.groupby(cols_5)[value_5].sum().reset_index()
df_5 = df_5[df_5["año"] == 2025]
order_regions = ['Arica y Parinacota',
                 'Tarapacá',
                 'Antofagasta', 
                 'Atacama',
                 'Coquimbo', 
                 'Valparaíso',
                 'Metropolitana',
                 "Lib. Gral. B. O'Higgins",
                 'Maule',   
                 'Ñuble',
                 'Biobío',
                 'La Araucanía',
                 'Los Ríos',
                 'Los Lagos',
                 'Aysén',
                 'Magallanes']

chart_5 = alt.Chart(df_5).mark_bar().encode(
    x = alt.X("total matrícula").title("Total enrolled"),
    y = alt.Y("región", sort= order_regions).title("Regions of Chile (South → North)")
).properties(
    title="Total enrolled university students by region for 2025",
    width = 400,
    height = 400
)
chart_5

alt.Chart(...)

# **Sixth Chart: Heat Map for career length and Study field for 2025**

The following Chart shows how long are careers per field study for Professional
Careers of Undergrads in 2025

In [12]:
x = "área del conocimiento"
y = "duración total de carrera"

df_6 = raw[(raw["año"] == 2025) & 
           (raw["clasificación institución nivel 1"] == "Universidades") &
           (raw["nivel global"] == "Pregrado")]
df_6 = df_6[["duración total de carrera", "área del conocimiento"]]
df_6 = (
    df_6.groupby([x,y])[y]
    .count()
    .reset_index(name="count")
)
df_6["área del conocimiento"] = df_6.apply(translate_area_del_conocimiento, axis=1)

chart_6 = (
    alt.Chart(df_6)
    .mark_rect()
    .encode(
        x=alt.X(x, title="Field Study"),
        y=alt.Y(y, type="ordinal", sort="descending", title="Total Length of Career (Semesters)"),
        color=alt.Color("count", scale=alt.Scale(scheme="blues", type="log"),
            title="Number of careers"
        ),
        tooltip=[x, y, "count"]
    )
    .properties(
        title="Distribution of length of careers by field study for Professional Careers in 2025"
    )
)
chart_6


alt.Chart(...)

# **Seventh Chart: Share of Men and Women per degree type in 2025**

The following Chart shows what's the distribution of men and women per degree type in 2025

In [13]:
cols_7 = ["año","carrera clasificación nivel 2"]
df_7 = raw[raw["año"] == 2025]
df_7 = df_7.groupby(cols_7)[["total matrícula hombres", "total matrícula mujeres"]].sum().reset_index()

df_7

df_7 = df_7.melt(id_vars= cols_7,
                 value_vars=["total matrícula hombres", "total matrícula mujeres"],
                 var_name="enrolled type",
                 value_name="total matrícula")

men = "total matrícula hombres"
women = "total matrícula mujeres"

df_7["enrolled type"] = df_7["enrolled type"].apply(lambda x: "Men" if x == men else "Women")
print(df_7["carrera clasificación nivel 2"].unique())
df_7["carrera clasificación nivel 2"] = df_7.apply(translates_degree, axis=1)

rule_50 = pd.DataFrame({"y": [0.50]})
line_50 = alt.Chart(rule_50).mark_rule(color="black").encode(
    y = "y:Q").properties(
        title="50%"
    )

order = [
    "Technical (Undergrad)",
    "Professionals (Undergrad)",
    "Diplomas (Grad)",
    "Masters (Grad)",
    "PhD (Grad)"
]
print(df_7["carrera clasificación nivel 2"].unique())
chart_7 = alt.Chart(df_7).mark_bar().encode(
    x = alt.X("carrera clasificación nivel 2:N",
              sort=order,  # <–– this enforces your custom order
              axis=alt.Axis(title="Degree Type")),
    y = alt.Y("sum(total matrícula)",
              stack="normalize",
              axis=alt.Axis(title="Total enrolled")),
    color = alt.Color("enrolled type",
                      legend=alt.Legend(title="Gender"))
).properties(
    title = "Share of Men and Women per degree in 2025",
    width = 400,
    height = 400
)

chart_7 + line_50

['Carreras Profesionales' 'Carreras Técnicas' 'Doctorado' 'Magister'
 'Postítulo']
['Professionals (Undergrad)' 'Technical (Undergrad)' 'PhD (Grad)'
 'Masters (Grad)' 'Diplomas (Grad)']


alt.LayerChart(...)

# **Eightth Chart: Evolution of spending  of % GDP per education type**

The following Chart shows how much Chile has been spending in primary, secondary,
and tertiary education as part of its GDP for period 2007-2021

In [26]:
# loading data
# url = https://ourworldindata.org/grapher/education-spending?country=%7ECHL&overlay=download-data&spending_type=gdp_share&level=level_side_by_side
spend = pd.read_csv("../data/raw/chile_spending.csv")

new_names = {
 'Year': 'year',
 'Government spending on tertiary education as share of GDP': 'Tertiary',
 'Government spending on upper secondary education as share of GDP': 'secondary_1',
 'Government spending on lower secondary education as share of GDP': 'secondary_2',
 'Government spending on primary education as share of GDP': 'Primary',
 'Government spending on pre-primary education as share of GDP': 'Pre-Primary'}
spend.rename(columns=new_names, inplace=True)
spend["Secondary"] = spend["secondary_1"] + spend["secondary_2"]
spend.drop(columns=["Entity", "Code", "secondary_1", "secondary_2"], inplace=True)

In [27]:
spend_long = spend.melt(id_vars=["year"],
                        var_name = "level",
                        value_vars = ["Pre-Primary", "Primary",
                                                        "Secondary", "Tertiary"])

In [35]:
chart_8 = alt.Chart(spend_long).mark_line().encode(
    x = alt.X("year:N").title("Year"),
    y = alt.Y("value").title("% of GDP"),
    color = alt.Color("level").title("Education Level")).properties(
        title = "Spend on education level as % of GDP",
        width = 500,
        height = 400
    )
chart_8

alt.Chart(...)